#  <center> Workflow </center>

![img](https://i.imgur.com/1IsBI7H.png)

N.B. 
* You need to run this notebook after turning on the GPU
* The database is cleaned and there's no missing values, that's why there's no scope for imputation
* Scaling didn't improve the score, that's why this step is eliminated
* LGBM performs the best out of all other algorithms, I removed low scored models as they were killing times
* As the dataset is imbalanced, I've used stratification 

In [1]:
# Count the time span of the notebook
import datetime
start_time = datetime.datetime.now()

# 1. Import Libraries

In [2]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from optuna.visualization import plot_optimization_history, plot_param_importances
from IPython.display import display
from pandas import set_option
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input,Dense,Dropout
from tensorflow.keras import Model
from  tensorflow.keras.regularizers import l2


import lightgbm as lgb
import numpy as np
import pandas as pd
import optuna
import tensorflow as tf


# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Show all the columns
pd.set_option('display.max_columns', None) 

### Set Constants

In [3]:
# Set seed to make the codebase reproducible
SEED = 42
np.random.seed(SEED)
batch_size = 128
epochs = 100
K = 15
num_folds = K

# Fetch Data

In [4]:
# Consider first column as index
train = pd.read_csv('../input/tabular-playground-series-mar-2021/train.csv', index_col=0)
test = pd.read_csv('../input/tabular-playground-series-mar-2021/test.csv', index_col=0)
train.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,A,I,A,B,B,BI,A,S,Q,A,LO,A,A,A,A,B,D,D,B,0.629858,0.855349,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0
1,A,I,A,A,E,BI,K,W,AD,F,HJ,A,B,A,B,D,B,D,B,0.370727,0.328929,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,0.493729,0
2,A,K,A,A,E,BI,A,E,BM,L,DJ,A,B,A,A,B,D,D,B,0.502272,0.322749,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0
3,A,K,A,C,E,BI,A,Y,AD,F,KV,A,A,A,A,B,D,D,B,0.934242,0.707663,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,0.934242,0
4,A,I,G,B,E,BI,C,G,Q,A,DP,A,A,A,B,B,B,D,B,0.254427,0.274514,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,0.328960,1


In [5]:
print("This dataset has {:.1f}".format(100*train.isna().to_numpy().sum()/(train.shape[0]*train.shape[1])) + "% missing values")

This dataset has 0.0% missing values


In [6]:
# Row and Column size of the training dataset
train.shape

(300000, 31)

# 2. Feature Engineering

In [7]:
# Show the number of missing values
missing = train.isnull().sum()
print(missing)

cat0      0
cat1      0
cat2      0
cat3      0
cat4      0
cat5      0
cat6      0
cat7      0
cat8      0
cat9      0
cat10     0
cat11     0
cat12     0
cat13     0
cat14     0
cat15     0
cat16     0
cat17     0
cat18     0
cont0     0
cont1     0
cont2     0
cont3     0
cont4     0
cont5     0
cont6     0
cont7     0
cont8     0
cont9     0
cont10    0
target    0
dtype: int64


# Remove Outliers

### dropna()

In [8]:
def remove_outliers_using_quantiles(qu_dataset, qu_field, qu_fence):
    a = qu_dataset[qu_field].describe()
    
    iqr = a["75%"] - a["25%"]
    print("interquartile range:", iqr)
    
    upper_inner_fence = a["75%"] + 1.5 * iqr
    lower_inner_fence = a["25%"] - 1.5 * iqr
    print("upper_inner_fence:", upper_inner_fence)
    print("lower_inner_fence:", lower_inner_fence)
    
    upper_outer_fence = a["75%"] + 3 * iqr
    lower_outer_fence = a["25%"] - 3 * iqr
    print("upper_outer_fence:", upper_outer_fence)
    print("lower_outer_fence:", lower_outer_fence)
    
    count_over_upper = len(qu_dataset[qu_dataset[qu_field]>upper_inner_fence])
    count_under_lower = len(qu_dataset[qu_dataset[qu_field]<lower_inner_fence])
    percentage = 100 * (count_under_lower + count_over_upper) / a["count"]
    print("percentage of records out of inner fences: %.2f"% (percentage))
    
    count_over_upper = len(qu_dataset[qu_dataset[qu_field]>upper_outer_fence])
    count_under_lower = len(qu_dataset[qu_dataset[qu_field]<lower_outer_fence])
    percentage = 100 * (count_under_lower + count_over_upper) / a["count"]
    print("percentage of records out of outer fences: %.2f"% (percentage))
    
    if qu_fence == "inner":
        output_dataset = qu_dataset[qu_dataset[qu_field]<=upper_inner_fence]
        output_dataset = output_dataset[output_dataset[qu_field]>=lower_inner_fence]
    elif qu_fence == "outer":
        output_dataset = qu_dataset[qu_dataset[qu_field]<=upper_outer_fence]
        output_dataset = output_dataset[output_dataset[qu_field]>=lower_outer_fence]
    else:
        output_dataset = qu_dataset
    
    print("length of input dataframe:", len(qu_dataset))
    print("length of new dataframe after outlier removal:", len(output_dataset))
    
    return output_dataset

# Drop the outliers rows
train.dropna(inplace=True)
new_dataset = remove_outliers_using_quantiles(train, "target", "inner")

# the dataset is actually already cleaned, hence no outliers

interquartile range: 1.0
upper_inner_fence: 2.5
lower_inner_fence: -1.5
upper_outer_fence: 4.0
lower_outer_fence: -3.0
percentage of records out of inner fences: 0.00
percentage of records out of outer fences: 0.00
length of input dataframe: 300000
length of new dataframe after outlier removal: 300000


In [9]:
# Show is there any imbalance in the target value, actually there's a huge imbalance,
# so stratification is needed
train['target'].value_counts()

0    220539
1     79461
Name: target, dtype: int64

# Denoising AutoEncoder (DAE)

In [10]:
# Create separate variables for both categorical and continous data
categorical_cols = ['cat'+str(i) for i in range(19)]
continous_cols = ['cont'+str(i) for i in range(11)]

In [11]:
cols=categorical_cols+continous_cols
train_objs_num = len(train)

# Just combining both train and test sets columns so that encoding categorical data becomes easy 
dataset = pd.concat(objs=[train[cols], test[cols]], axis=0)

# Encode only the categories, though LGBM doesn't need that conversion
# As I'll compare with other models, I'm converting them anyway
dataset_preprocessed = pd.get_dummies(dataset,columns=categorical_cols)

# Separate train and test set again
train_preprocessed = dataset_preprocessed[:train_objs_num]
test_preprocessed = dataset_preprocessed[train_objs_num:]

train_preprocessed.head()

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cat0_A,cat0_B,cat1_A,cat1_B,cat1_C,cat1_D,cat1_E,cat1_F,cat1_G,cat1_H,cat1_I,cat1_J,cat1_K,cat1_L,cat1_M,cat1_N,cat1_O,cat2_A,cat2_B,cat2_C,cat2_D,cat2_E,cat2_F,cat2_G,cat2_H,cat2_I,cat2_J,cat2_K,cat2_L,cat2_M,cat2_N,cat2_O,cat2_Q,cat2_R,cat2_S,cat2_U,cat3_A,cat3_B,cat3_C,cat3_D,cat3_E,cat3_F,cat3_G,cat3_H,cat3_I,cat3_J,cat3_K,cat3_L,cat3_N,cat4_A,cat4_B,cat4_C,cat4_D,cat4_E,cat4_F,cat4_G,cat4_H,cat4_I,cat4_J,cat4_K,cat4_L,cat4_M,cat4_N,cat4_O,cat4_P,cat4_Q,cat4_R,cat4_S,cat4_T,cat5_A,cat5_AA,cat5_AB,cat5_AE,cat5_AF,cat5_AG,cat5_AH,cat5_AI,cat5_AJ,cat5_AK,cat5_AL,cat5_AM,cat5_AN,cat5_AO,cat5_AP,cat5_AQ,cat5_AR,cat5_AS,cat5_AT,cat5_AU,cat5_AV,cat5_AW,cat5_AX,cat5_AY,cat5_B,cat5_BA,cat5_BB,cat5_BC,cat5_BD,cat5_BE,cat5_BF,cat5_BG,cat5_BH,cat5_BI,cat5_BJ,cat5_BK,cat5_BL,cat5_BM,cat5_BN,cat5_BO,cat5_BP,cat5_BQ,cat5_BR,cat5_BS,cat5_BT,cat5_BU,cat5_BV,cat5_BX,cat5_BY,cat5_C,cat5_CA,cat5_CB,cat5_CC,cat5_CD,cat5_CE,cat5_CF,cat5_CG,cat5_CH,cat5_CI,cat5_CJ,cat5_CK,cat5_CL,cat5_D,cat5_E,cat5_F,cat5_G,cat5_H,cat5_I,cat5_J,cat5_K,cat5_L,cat5_M,cat5_N,cat5_O,cat5_P,cat5_Q,cat5_R,cat5_S,cat5_T,cat5_U,cat5_V,cat5_X,cat5_Y,cat5_ZZ,cat6_A,cat6_B,cat6_C,cat6_D,cat6_E,cat6_F,cat6_G,cat6_I,cat6_K,cat6_M,cat6_O,cat6_Q,cat6_S,cat6_U,cat6_W,cat6_Y,cat7_A,cat7_AA,cat7_AB,cat7_AC,cat7_AD,cat7_AE,cat7_AF,cat7_AG,cat7_AH,cat7_AI,cat7_AJ,cat7_AK,cat7_AL,cat7_AM,cat7_AN,cat7_AO,cat7_AP,cat7_AQ,cat7_AR,cat7_AS,cat7_AT,cat7_AU,cat7_AV,cat7_AW,cat7_AX,cat7_AY,cat7_B,cat7_BA,cat7_C,cat7_D,cat7_E,cat7_F,cat7_G,cat7_H,cat7_I,cat7_J,cat7_K,cat7_L,cat7_M,cat7_N,cat7_O,cat7_P,cat7_Q,cat7_R,cat7_S,cat7_T,cat7_U,cat7_V,cat7_W,cat7_X,cat7_Y,cat8_A,cat8_AB,cat8_AC,cat8_AD,cat8_AE,cat8_AF,cat8_AG,cat8_AH,cat8_AI,cat8_AJ,cat8_AK,cat8_AL,cat8_AM,cat8_AN,cat8_AO,cat8_AP,cat8_AQ,cat8_AR,cat8_AS,cat8_AT,cat8_AU,cat8_AV,cat8_AW,cat8_AX,cat8_AY,cat8_B,cat8_BA,cat8_BB,cat8_BC,cat8_BD,cat8_BE,cat8_BF,cat8_BG,cat8_BH,cat8_BI,cat8_BJ,cat8_BK,cat8_BL,cat8_BM,cat8_BN,cat8_BO,cat8_C,cat8_E,cat8_F,cat8_G,cat8_H,cat8_I,cat8_J,cat8_K,cat8_L,cat8_M,cat8_N,cat8_O,cat8_P,cat8_Q,cat8_S,cat8_T,cat8_U,cat8_V,cat8_X,cat8_Y,cat9_A,cat9_B,cat9_C,cat9_D,cat9_E,cat9_F,cat9_G,cat9_I,cat9_J,cat9_L,cat9_N,cat9_O,cat9_Q,cat9_R,cat9_S,cat9_U,cat9_V,cat9_W,cat9_X,cat10_AA,cat10_AB,cat10_AC,cat10_AD,cat10_AE,cat10_AF,cat10_AG,cat10_AH,cat10_AJ,cat10_AK,cat10_AL,cat10_AM,cat10_AN,cat10_AP,cat10_AR,cat10_AS,cat10_AT,cat10_AU,cat10_AV,cat10_AW,cat10_AY,cat10_BA,cat10_BB,cat10_BC,cat10_BD,cat10_BF,cat10_BG,cat10_BI,cat10_BL,cat10_BM,cat10_BO,cat10_BP,cat10_BQ,cat10_BS,cat10_BT,cat10_BU,cat10_BV,cat10_BW,cat10_BX,cat10_BY,cat10_C,cat10_CA,cat10_CB,cat10_CC,cat10_CD,cat10_CF,cat10_CG,cat10_CH,cat10_CI,cat10_CJ,cat10_CK,cat10_CL,cat10_CM,cat10_CN,cat10_CO,cat10_CP,cat10_CQ,cat10_CR,cat10_CS,cat10_CT,cat10_CU,cat10_CW,cat10_CX,cat10_CY,cat10_D,cat10_DA,cat10_DC,cat10_DD,cat10_DE,cat10_DF,cat10_DG,cat10_DH,cat10_DI,cat10_DJ,cat10_DK,cat10_DL,cat10_DM,cat10_DN,cat10_DO,cat10_DP,cat10_DQ,cat10_DR,cat10_DS,cat10_DT,cat10_DU,cat10_DV,cat10_DW,cat10_DX,cat10_DY,cat10_E,cat10_EA,cat10_EB,cat10_EC,cat10_ED,cat10_EE,cat10_EF,cat10_EG,cat10_EH,cat10_EI,cat10_EJ,cat10_EK,cat10_EL,cat10_EN,cat10_EO,cat10_EP,cat10_EQ,cat10_ES,cat10_EU,cat10_EV,cat10_EW,cat10_EY,cat10_F,cat10_FA,cat10_FB,cat10_FC,cat10_FD,cat10_FE,cat10_FF,cat10_FG,cat10_FH,cat10_FI,cat10_FJ,cat10_FK,cat10_FL,cat10_FM,cat10_FN,cat10_FO,cat10_FP,cat10_FQ,cat10_FR,cat10_FS,cat10_FT,cat10_FV,cat10_FW,cat10_FX,cat10_G,cat10_GA,cat10_GB,cat10_GC,cat10_GD,cat10_GE,cat10_GF,cat10_GG,cat10_GH,cat10_GI,cat10_GJ,cat10_GK,cat10_GL,cat10_GM,cat10_GN,cat10_GQ,cat10_GR,cat10_GS,cat10_GT,cat10_GU,cat10_GV,cat10_GW,cat10_GX,cat10_GY,cat10_HA,cat10_HB,cat10_HC,cat10_HE,cat10_HF,cat10_HG,cat10_HH,cat10_HI,cat10_HJ,cat10_HK,cat10_HL,cat10_HM,cat10_HN,cat10_HO,cat10_HP,cat10_HQ,cat10_HR,cat10_HU,cat10_HV,cat10_HW,cat10_HX,cat10_HY,cat10_I,cat10_IA,cat10_IB,cat10_IC,cat10_ID,cat10_IE,cat10_IF,cat10_IG,cat10_IH,cat

In [12]:
def get_DAE():
    # The dataset contains 11 independent variables
    inputs = Input((11,))
    x = Dense(1500, activation='relu')(inputs) # 1500 original
    x = Dense(1500, activation='relu', name="feature")(x) # 1500 original
    x = Dense(1500, activation='relu')(x) # 1500 original
    outputs = Dense(11, activation='relu')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse')

    return model

In [13]:
# Here we don't need any target value, as DAE slightly corrupts the input data but still, 
# maintain the uncorrupted data as our target output.
alldata = pd.concat([train[continous_cols],test[continous_cols]],axis=0)
print(alldata.shape)
autoencoder = get_DAE()
autoencoder.fit(alldata[continous_cols], alldata[continous_cols],
                    epochs=20,
                    batch_size=256,
                    shuffle=True
                    )

(500000, 11)
Epoch 1/20
1954/1954 [==============================] - 8s 3ms/step - loss: 0.0338
Epoch 2/20
1954/1954 [==============================] - 6s 3ms/step - loss: 0.0248
Epoch 3/20
1954/1954 [==============================] - 6s 3ms/step - loss: 0.0248
Epoch 4/20
1954/1954 [==============================] - 6s 3ms/step - loss: 0.0248
Epoch 5/20
1954/1954 [==============================] - 6s 3ms/step - loss: 0.0228
Epoch 6/20
1954/1954 [==============================] - 7s 3ms/step - loss: 1.9216e-05
Epoch 7/20
1954/1954 [==============================] - 6s 3ms/step - loss: 2.2298e-05
Epoch 8/20
1954/1954 [==============================] - 6s 3ms/step - loss: 2.0879e-05
Epoch 9/20
1954/1954 [==============================] - 6s 3ms/step - loss: 1.9484e-05
Epoch 10/20
1954/1954 [==============================] - 6s 3ms/step - loss: 1.3397e-05
Epoch 11/20
1954/1954 [==============================] - 6s 3ms/step - loss: 1.4023e-05
Epoch 12/20
1954/1954 [=========================

In [14]:
# Create corrupted data for the corresponding input
test_denoised = test_preprocessed.copy()
test_denoised[continous_cols] = autoencoder.predict(test_denoised[continous_cols])
train_denoised = train_preprocessed.copy()
train_denoised[continous_cols] = autoencoder.predict(train_denoised[continous_cols])

In [15]:
train_denoised['target'] = train.target

In [16]:
X = train_denoised.drop(['target'], axis=1)
Y = train_denoised.target
X_TEST = test_denoised

In [17]:
train.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,A,I,A,B,B,BI,A,S,Q,A,LO,A,A,A,A,B,D,D,B,0.629858,0.855349,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0
1,A,I,A,A,E,BI,K,W,AD,F,HJ,A,B,A,B,D,B,D,B,0.370727,0.328929,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,0.493729,0
2,A,K,A,A,E,BI,A,E,BM,L,DJ,A,B,A,A,B,D,D,B,0.502272,0.322749,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0
3,A,K,A,C,E,BI,A,Y,AD,F,KV,A,A,A,A,B,D,D,B,0.934242,0.707663,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,0.934242,0
4,A,I,G,B,E,BI,C,G,Q,A,DP,A,A,A,B,B,B,D,B,0.254427,0.274514,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,0.328960,1


In [18]:
X.head()

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cat0_A,cat0_B,cat1_A,cat1_B,cat1_C,cat1_D,cat1_E,cat1_F,cat1_G,cat1_H,cat1_I,cat1_J,cat1_K,cat1_L,cat1_M,cat1_N,cat1_O,cat2_A,cat2_B,cat2_C,cat2_D,cat2_E,cat2_F,cat2_G,cat2_H,cat2_I,cat2_J,cat2_K,cat2_L,cat2_M,cat2_N,cat2_O,cat2_Q,cat2_R,cat2_S,cat2_U,cat3_A,cat3_B,cat3_C,cat3_D,cat3_E,cat3_F,cat3_G,cat3_H,cat3_I,cat3_J,cat3_K,cat3_L,cat3_N,cat4_A,cat4_B,cat4_C,cat4_D,cat4_E,cat4_F,cat4_G,cat4_H,cat4_I,cat4_J,cat4_K,cat4_L,cat4_M,cat4_N,cat4_O,cat4_P,cat4_Q,cat4_R,cat4_S,cat4_T,cat5_A,cat5_AA,cat5_AB,cat5_AE,cat5_AF,cat5_AG,cat5_AH,cat5_AI,cat5_AJ,cat5_AK,cat5_AL,cat5_AM,cat5_AN,cat5_AO,cat5_AP,cat5_AQ,cat5_AR,cat5_AS,cat5_AT,cat5_AU,cat5_AV,cat5_AW,cat5_AX,cat5_AY,cat5_B,cat5_BA,cat5_BB,cat5_BC,cat5_BD,cat5_BE,cat5_BF,cat5_BG,cat5_BH,cat5_BI,cat5_BJ,cat5_BK,cat5_BL,cat5_BM,cat5_BN,cat5_BO,cat5_BP,cat5_BQ,cat5_BR,cat5_BS,cat5_BT,cat5_BU,cat5_BV,cat5_BX,cat5_BY,cat5_C,cat5_CA,cat5_CB,cat5_CC,cat5_CD,cat5_CE,cat5_CF,cat5_CG,cat5_CH,cat5_CI,cat5_CJ,cat5_CK,cat5_CL,cat5_D,cat5_E,cat5_F,cat5_G,cat5_H,cat5_I,cat5_J,cat5_K,cat5_L,cat5_M,cat5_N,cat5_O,cat5_P,cat5_Q,cat5_R,cat5_S,cat5_T,cat5_U,cat5_V,cat5_X,cat5_Y,cat5_ZZ,cat6_A,cat6_B,cat6_C,cat6_D,cat6_E,cat6_F,cat6_G,cat6_I,cat6_K,cat6_M,cat6_O,cat6_Q,cat6_S,cat6_U,cat6_W,cat6_Y,cat7_A,cat7_AA,cat7_AB,cat7_AC,cat7_AD,cat7_AE,cat7_AF,cat7_AG,cat7_AH,cat7_AI,cat7_AJ,cat7_AK,cat7_AL,cat7_AM,cat7_AN,cat7_AO,cat7_AP,cat7_AQ,cat7_AR,cat7_AS,cat7_AT,cat7_AU,cat7_AV,cat7_AW,cat7_AX,cat7_AY,cat7_B,cat7_BA,cat7_C,cat7_D,cat7_E,cat7_F,cat7_G,cat7_H,cat7_I,cat7_J,cat7_K,cat7_L,cat7_M,cat7_N,cat7_O,cat7_P,cat7_Q,cat7_R,cat7_S,cat7_T,cat7_U,cat7_V,cat7_W,cat7_X,cat7_Y,cat8_A,cat8_AB,cat8_AC,cat8_AD,cat8_AE,cat8_AF,cat8_AG,cat8_AH,cat8_AI,cat8_AJ,cat8_AK,cat8_AL,cat8_AM,cat8_AN,cat8_AO,cat8_AP,cat8_AQ,cat8_AR,cat8_AS,cat8_AT,cat8_AU,cat8_AV,cat8_AW,cat8_AX,cat8_AY,cat8_B,cat8_BA,cat8_BB,cat8_BC,cat8_BD,cat8_BE,cat8_BF,cat8_BG,cat8_BH,cat8_BI,cat8_BJ,cat8_BK,cat8_BL,cat8_BM,cat8_BN,cat8_BO,cat8_C,cat8_E,cat8_F,cat8_G,cat8_H,cat8_I,cat8_J,cat8_K,cat8_L,cat8_M,cat8_N,cat8_O,cat8_P,cat8_Q,cat8_S,cat8_T,cat8_U,cat8_V,cat8_X,cat8_Y,cat9_A,cat9_B,cat9_C,cat9_D,cat9_E,cat9_F,cat9_G,cat9_I,cat9_J,cat9_L,cat9_N,cat9_O,cat9_Q,cat9_R,cat9_S,cat9_U,cat9_V,cat9_W,cat9_X,cat10_AA,cat10_AB,cat10_AC,cat10_AD,cat10_AE,cat10_AF,cat10_AG,cat10_AH,cat10_AJ,cat10_AK,cat10_AL,cat10_AM,cat10_AN,cat10_AP,cat10_AR,cat10_AS,cat10_AT,cat10_AU,cat10_AV,cat10_AW,cat10_AY,cat10_BA,cat10_BB,cat10_BC,cat10_BD,cat10_BF,cat10_BG,cat10_BI,cat10_BL,cat10_BM,cat10_BO,cat10_BP,cat10_BQ,cat10_BS,cat10_BT,cat10_BU,cat10_BV,cat10_BW,cat10_BX,cat10_BY,cat10_C,cat10_CA,cat10_CB,cat10_CC,cat10_CD,cat10_CF,cat10_CG,cat10_CH,cat10_CI,cat10_CJ,cat10_CK,cat10_CL,cat10_CM,cat10_CN,cat10_CO,cat10_CP,cat10_CQ,cat10_CR,cat10_CS,cat10_CT,cat10_CU,cat10_CW,cat10_CX,cat10_CY,cat10_D,cat10_DA,cat10_DC,cat10_DD,cat10_DE,cat10_DF,cat10_DG,cat10_DH,cat10_DI,cat10_DJ,cat10_DK,cat10_DL,cat10_DM,cat10_DN,cat10_DO,cat10_DP,cat10_DQ,cat10_DR,cat10_DS,cat10_DT,cat10_DU,cat10_DV,cat10_DW,cat10_DX,cat10_DY,cat10_E,cat10_EA,cat10_EB,cat10_EC,cat10_ED,cat10_EE,cat10_EF,cat10_EG,cat10_EH,cat10_EI,cat10_EJ,cat10_EK,cat10_EL,cat10_EN,cat10_EO,cat10_EP,cat10_EQ,cat10_ES,cat10_EU,cat10_EV,cat10_EW,cat10_EY,cat10_F,cat10_FA,cat10_FB,cat10_FC,cat10_FD,cat10_FE,cat10_FF,cat10_FG,cat10_FH,cat10_FI,cat10_FJ,cat10_FK,cat10_FL,cat10_FM,cat10_FN,cat10_FO,cat10_FP,cat10_FQ,cat10_FR,cat10_FS,cat10_FT,cat10_FV,cat10_FW,cat10_FX,cat10_G,cat10_GA,cat10_GB,cat10_GC,cat10_GD,cat10_GE,cat10_GF,cat10_GG,cat10_GH,cat10_GI,cat10_GJ,cat10_GK,cat10_GL,cat10_GM,cat10_GN,cat10_GQ,cat10_GR,cat10_GS,cat10_GT,cat10_GU,cat10_GV,cat10_GW,cat10_GX,cat10_GY,cat10_HA,cat10_HB,cat10_HC,cat10_HE,cat10_HF,cat10_HG,cat10_HH,cat10_HI,cat10_HJ,cat10_HK,cat10_HL,cat10_HM,cat10_HN,cat10_HO,cat10_HP,cat10_HQ,cat10_HR,cat10_HU,cat10_HV,cat10_HW,cat10_HX,cat10_HY,cat10_I,cat10_IA,cat10_IB,cat10_IC,cat10_ID,cat10_IE,cat10_IF,cat10_IG,cat10_IH,cat

# Train Test Split

In [19]:
# As the labels are not equal, stratification is employed
X_train, X_test, y_train, y_test =train_test_split(X,train['target'],
                                                   test_size=0.20,
                                                   random_state=42,
                                                   stratify=train['target'])

# 3. Models Implementation

### Deep Neural Network

In [20]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)
# Early stopping should be based on loss, not on accuracy
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=0, save_best_only=True)

In [21]:
model = keras.Sequential([
    ## reshaping the input entries
    # there're 30 independent variables 
    keras.layers.Dense(50, input_shape=(X.shape[1],), activation='relu'),  
    keras.layers.Dropout(0.50),    ## to avoid overfitting and underfiting

    ## creating the hidden layer
    keras.layers.Dense(100,activation='relu'),
    keras.layers.Dropout(0.70),    ##  to avoid overfitting and underfiting
    
    keras.layers.Dense(150,activation='relu'),
    keras.layers.Dropout(0.70),     ## to avoid overfitting and underfiting
 
    # sigmoid as this is a binary classification problem
    ## final neural layer
    keras.layers.Dense(1,activation='sigmoid')
    
])


# as an optimizer, adams give promising performance
model.compile(optimizer='adam',
             loss='binary_crossentropy',  ## since output in 0 or 1
             metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.AUC()])

model.fit(X,Y,epochs=epochs, 
          batch_size=batch_size,
          callbacks=[annealer,es,mc],
          validation_split=0.1,
          shuffle=True,
         )

y_pred_cnn = model.predict(X_TEST)

cnn_score = model.evaluate(X, Y)[1]
cnn_score

Epoch 1/100
2110/2110 [==============================] - 11s 5ms/step - loss: 0.4286 - binary_accuracy: 0.8096 - auc: 0.8315 - val_loss: 0.3658 - val_binary_accuracy: 0.8456 - val_auc: 0.8821
Epoch 2/100
2110/2110 [==============================] - 9s 4ms/step - loss: 0.3798 - binary_accuracy: 0.8396 - auc: 0.8723 - val_loss: 0.3629 - val_binary_accuracy: 0.8462 - val_auc: 0.8834
Epoch 3/100
2110/2110 [==============================] - 9s 4ms/step - loss: 0.3741 - binary_accuracy: 0.8428 - auc: 0.8751 - val_loss: 0.3628 - val_binary_accuracy: 0.8479 - val_auc: 0.8842
Epoch 4/100
2110/2110 [==============================] - 9s 4ms/step - loss: 0.3705 - binary_accuracy: 0.8434 - auc: 0.8780 - val_loss: 0.3577 - val_binary_accuracy: 0.8477 - val_auc: 0.8849
Epoch 5/100
2110/2110 [==============================] - 9s 4ms/step - loss: 0.3686 - binary_accuracy: 0.8447 - auc: 0.8795 - val_loss: 0.3631 - val_binary_accuracy: 0.8474 - val_auc: 0.8849
Epoch 6/100
2110/2110 [=====================

0.8541100025177002

# Try different Machine Learning Base Models

In [22]:
# Spot-Check Algorithms
def GetBasedModel():
    basedModels = []
    basedModels.append(('NB'   , GaussianNB()))
    basedModels.append(('XGB'   , XGBClassifier(verbosity = 0)))
    basedModels.append(('LGB'   , LGBMClassifier()))
#     basedModels.append(('LR'   , LogisticRegression()))
#     basedModels.append(('LDA'  , LinearDiscriminantAnalysis()))
#     basedModels.append(('KNN'  , KNeighborsClassifier()))
#     basedModels.append(('CART' , DecisionTreeClassifier()))
#     basedModels.append(('SVM'  , SVC(probability=True)))
#     basedModels.append(('AB'   , AdaBoostClassifier()))
#     basedModels.append(('GBM'  , GradientBoostingClassifier()))
#     basedModels.append(('RF'   , RandomForestClassifier()))
#     basedModels.append(('ET'   , ExtraTreesClassifier()))
#     basedModels.append(('CBC'   , CatBoostClassifier()))


    return basedModels

In [23]:
def BasedLine2(X_train, y_train,models):
    # Test options and evaluation metric
    scoring = 'roc_auc'

    results = []
    names = []
    for name, model in models:
        # stratification is needed for imbalanced target of dataframe
        kfold = StratifiedKFold(n_splits=num_folds)
        cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
        
    return names, results

In [24]:
models = GetBasedModel()

# Analyze all the base models
names,results = BasedLine2(X_train, y_train,models)

NB: 0.542484 (0.001661)
XGB: 0.889535 (0.002014)
LGB: 0.889727 (0.002093)


In [25]:
# Compare all the models according to the baseline score

def ScoreDataFrame(names,results):
    def floatingDecimals(f_val, dec=3):
        prc = "{:."+str(dec)+"f}" 
    
        return float(prc.format(f_val))

    scores = []
    for r in results:
        scores.append(floatingDecimals(r.mean(),4))

    scoreDataFrame = pd.DataFrame({'Model':names, 'Score': scores})
    return scoreDataFrame

In [26]:
basedLineScore = ScoreDataFrame(names,results)
basedLineScore

,Model,Score
0,NB,0.5425
1,XGB,0.8895
2,LGB,0.8897


# 4. Scaling the Dataframe

In [27]:
# Standard and MinMax scalers are considered
# Let's see which one performs better
def GetScaledModel(nameOfScaler):
    
    if nameOfScaler == 'standard':
        scaler = StandardScaler()
    elif nameOfScaler =='minmax':
        scaler = MinMaxScaler()

    pipelines = []
    pipelines.append((nameOfScaler+'NB'  , Pipeline([('Scaler', scaler),('NB'  , GaussianNB())])))
    pipelines.append((nameOfScaler+'XGB'  , Pipeline([('Scaler', scaler),('XGB'  , XGBClassifier())])))
    pipelines.append((nameOfScaler+'LGB'  , Pipeline([('Scaler', scaler),('LGB'  , LGBMClassifier())])))
#     pipelines.append((nameOfScaler+'LR'  , Pipeline([('Scaler', scaler),('LR'  , LogisticRegression())])))
#     pipelines.append((nameOfScaler+'LDA' , Pipeline([('Scaler', scaler),('LDA' , LinearDiscriminantAnalysis())])))
#     pipelines.append((nameOfScaler+'KNN' , Pipeline([('Scaler', scaler),('KNN' , KNeighborsClassifier())])))
#     pipelines.append((nameOfScaler+'CART', Pipeline([('Scaler', scaler),('CART', DecisionTreeClassifier())])))
#     pipelines.append((nameOfScaler+'SVM' , Pipeline([('Scaler', scaler),('SVM' , SVC())])))
#     pipelines.append((nameOfScaler+'AB'  , Pipeline([('Scaler', scaler),('AB'  , AdaBoostClassifier())])  ))
#     pipelines.append((nameOfScaler+'GBM' , Pipeline([('Scaler', scaler),('GMB' , GradientBoostingClassifier())])  ))
#     pipelines.append((nameOfScaler+'RF'  , Pipeline([('Scaler', scaler),('RF'  , RandomForestClassifier())])  ))
#     pipelines.append((nameOfScaler+'ET'  , Pipeline([('Scaler', scaler),('ET'  , ExtraTreesClassifier())])  ))


    return pipelines 

### Standard Scaler

In [28]:
models = GetScaledModel('standard')
names,results = BasedLine2(X_train, y_train,models)
scaledScoreStandard = ScoreDataFrame(names,results)
compareModels = pd.concat([basedLineScore,
                           scaledScoreStandard], axis=1)
compareModels

standardNB: 0.529301 (0.001159)
standardXGB: 0.889535 (0.002014)
standardLGB: 0.889773 (0.002148)


,Model,Score,Model,Score
0,NB,0.5425,standardNB,0.5293
1,XGB,0.8895,standardXGB,0.8895
2,LGB,0.8897,standardLGB,0.8898


# MinMax Scaler

In [29]:
names,results = BasedLine2(X_train, y_train,models)
scaledScoreMinMax = ScoreDataFrame(names,results)
compareModels = pd.concat([basedLineScore,
                           scaledScoreStandard,
                          scaledScoreMinMax], axis=1)
compareModels

standardNB: 0.529301 (0.001159)
standardXGB: 0.889535 (0.002014)
standardLGB: 0.889773 (0.002148)


,Model,Score,Model,Score,Model,Score
0,NB,0.5425,standardNB,0.5293,standardNB,0.5293
1,XGB,0.8895,standardXGB,0.8895,standardXGB,0.8895
2,LGB,0.8897,standardLGB,0.8898,standardLGB,0.8898


## Functions for Kfold-Cross validaiton, training, evaluation and prediction

In [30]:
# Functions for KFold evaluation
def create(hyperparams):
    """Create LGBM Classifier for a given set of hyper-parameters."""
    model = LGBMClassifier(**hyperparams)
    return model

def fit(model, X, y):
    """Simple training of a given model."""
    model.fit(X, y)
    return model

def fit_with_stop(model, X, y, X_val, y_val, esr):
    """Advanced training with early stopping."""
    model.fit(X, y,
              eval_set=(X_val, y_val),
              early_stopping_rounds=esr, 
              verbose=200)
    return model

def evaluate(model, X, y):
    """Compute AUC for a given model."""
    yp = model.predict_proba(X)[:, 1]
    auc_score = roc_auc_score(y, yp)
    return auc_score

def kfold_evaluation(X, y, k, hyperparams, esr=100):
    """Run a KFlod evaluation."""
    scores = []
    
    print(f"\n------ {k}-fold evaluation -----")
    print(hyperparams)
    
    kf = StratifiedKFold(k)
    for i, (train_idx, test_idx) in enumerate(kf.split(X,y)):
        print(f"\n----- FOLD {i} -----")
        
        X_train = X.iloc[train_idx]
        y_train = y.iloc[train_idx]
        X_val = X.iloc[test_idx]
        y_val = y.iloc[test_idx]
        
        model = create(hyperparams)
        model = fit_with_stop(model, X_train, y_train, X_val, y_val, esr)
        train_score = evaluate(model, X_train, y_train)
        val_score = evaluate(model, X_val, y_val)
        scores.append((train_score, val_score))
        
        # Using AUC, as per competition's rules
        print(f"Fold {i} | Eval AUC: {val_score}")
        
        
    scores = pd.DataFrame(scores, columns=['train score', 'validation score'])
    
    return scores


def kfold_prediction(X, y, X_test, k, hyperparams, esr=100):
    """Make predictions with a bagged model based on KFold."""
    yp = np.zeros(len(X_test))
    
    print(f"\n------ {k}-fold evaluation -----")
    print(hyperparams)
    
    kf = KFold(k)
    for i, (train_idx, test_idx) in enumerate(kf.split(X)):
        print(f"\n----- FOLD {i} -----")
        X_train = X.iloc[train_idx]
        y_train = y.iloc[train_idx]
        X_val = X.iloc[test_idx]
        y_val = y.iloc[test_idx]
        
        model = create(hyperparams)
        model = fit_with_stop(model, X_train, y_train, X_val, y_val, esr)
        yp += model.predict_proba(X_test)[:, 1] / k
       
    
    return yp

# 5. Hyperparameter tuning (OPTUNA)

In [31]:
# Set default parameters
BEST_PARAMS = {
    'n_estimators': 10000, # Number of boosted trees to fit
    'learning_rate': 0.05, # Me
    'metric': 'auc', # Me
    'device_type': 'gpu'
}

In [32]:
# Objective function
# Scrutinize the best hyperparameter using this function
def objective(trial):
    # Search spaces
    hyperparams = {
    'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2),
    'max_depth': trial.suggest_int('max_depth', 6, 127),
    'num_leaves': trial.suggest_int('num_leaves', 31, 128),
    'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0),
    'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0),
    'random_state': 2021,
    'metric': 'auc',
    'n_estimators': 20000,
    'n_jobs': -1,
    'bagging_seed': 2021,
    'feature_fraction_seed': 2021,
    'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 0.9),
    'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
    'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
    'subsample': trial.suggest_float('subsample', 0.3, 0.9),
    'min_data_per_group': trial.suggest_int('min_data_per_group', 50, 200),
    'cat_smooth': trial.suggest_int('cat_smooth', 10, 100),
    'cat_l2': trial.suggest_int('cat_l2', 1, 20),
    }
    
    # Add BEST_PARAMS
    hyperparams.update(BEST_PARAMS)
    
    # Evaluation
    scores = kfold_evaluation(X, Y, K, hyperparams, 100)
    
    return scores['validation score'].mean()

In [33]:
# Optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)
# study.optimize(objective, timeout=3600*7)

[I 2021-04-14 00:44:23,341] A new study created in memory with name: no-name-06d77301-3141-4eaf-98ec-bca475164805



------ 15-fold evaluation -----
{'learning_rate': 0.05, 'max_depth': 62, 'num_leaves': 128, 'reg_alpha': 6.980317919660197, 'reg_lambda': 4.4886961691334575, 'random_state': 2021, 'metric': 'auc', 'n_estimators': 10000, 'n_jobs': -1, 'bagging_seed': 2021, 'feature_fraction_seed': 2021, 'colsample_bytree': 0.6399546685072549, 'min_child_samples': 292, 'subsample_freq': 2, 'subsample': 0.3760490859865253, 'min_data_per_group': 165, 'cat_smooth': 96, 'cat_l2': 11, 'device_type': 'gpu'}

----- FOLD 0 -----
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.89391
[400]	valid_0's auc: 0.894836
Early stopping, best iteration is:
[440]	valid_0's auc: 0.895092
Fold 0 | Eval AUC: 0.895091909224327

----- FOLD 1 -----
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.89653
[400]	valid_0's auc: 0.897481
Early stopping, best iteration is:
[427]	valid_0's auc: 0.897643
Fold 1 | Eval AUC: 0.897642595301898

----- FOLD 2 -----
Train

[I 2021-04-14 01:03:33,020] Trial 0 finished with value: 0.8941132052575577 and parameters: {'learning_rate': 0.0031107369271109724, 'max_depth': 62, 'num_leaves': 128, 'reg_alpha': 6.980317919660197, 'reg_lambda': 4.4886961691334575, 'colsample_bytree': 0.6399546685072549, 'min_child_samples': 292, 'subsample_freq': 2, 'subsample': 0.3760490859865253, 'min_data_per_group': 165, 'cat_smooth': 96, 'cat_l2': 11}. Best is trial 0 with value: 0.8941132052575577.


Fold 14 | Eval AUC: 0.8953384924273086

------ 15-fold evaluation -----
{'learning_rate': 0.05, 'max_depth': 16, 'num_leaves': 96, 'reg_alpha': 7.791831090627568, 'reg_lambda': 2.115876077924751, 'random_state': 2021, 'metric': 'auc', 'n_estimators': 10000, 'n_jobs': -1, 'bagging_seed': 2021, 'feature_fraction_seed': 2021, 'colsample_bytree': 0.6829919771493201, 'min_child_samples': 247, 'subsample_freq': 5, 'subsample': 0.7068570560006351, 'min_data_per_group': 180, 'cat_smooth': 91, 'cat_l2': 20, 'device_type': 'gpu'}

----- FOLD 0 -----
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.894477
[400]	valid_0's auc: 0.895941
[600]	valid_0's auc: 0.89598
Early stopping, best iteration is:
[641]	valid_0's auc: 0.896034
Fold 0 | Eval AUC: 0.8960344273541425

----- FOLD 1 -----
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.897035
[400]	valid_0's auc: 0.898763
[600]	valid_0's auc: 0.898814
Early stopping, best iterati

[I 2021-04-14 01:24:10,449] Trial 1 finished with value: 0.8951989092690847 and parameters: {'learning_rate': 0.004272638107599317, 'max_depth': 16, 'num_leaves': 96, 'reg_alpha': 7.791831090627568, 'reg_lambda': 2.115876077924751, 'colsample_bytree': 0.6829919771493201, 'min_child_samples': 247, 'subsample_freq': 5, 'subsample': 0.7068570560006351, 'min_data_per_group': 180, 'cat_smooth': 91, 'cat_l2': 20}. Best is trial 1 with value: 0.8951989092690847.


...

Fold 14 | Eval AUC: 0.8965956306538162

------ 15-fold evaluation -----
{'learning_rate': 0.05, 'max_depth': 88, 'num_leaves': 126, 'reg_alpha': 7.060944214852537, 'reg_lambda': 2.7970374277678913, 'random_state': 2021, 'metric': 'auc', 'n_estimators': 10000, 'n_jobs': -1, 'bagging_seed': 2021, 'feature_fraction_seed': 2021, 'colsample_bytree': 0.7563673780575333, 'min_child_samples': 9, 'subsample_freq': 4, 'subsample': 0.5395396760653526, 'min_data_per_group': 128, 'cat_smooth': 83, 'cat_l2': 5, 'device_type': 'gpu'}

----- FOLD 0 -----
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.894479
[400]	valid_0's auc: 0.895337
Early stopping, best iteration is:
[411]	valid_0's auc: 0.895415
Fold 0 | Eval AUC: 0.8954154251537435

----- FOLD 1 -----
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.897793
[400]	valid_0's auc: 0.898938
Early stopping, best iteration is:
[439]	valid_0's auc: 0.899022
Fold 1 | Eval AUC: 0.89

[I 2021-04-14 02:04:03,414] Trial 3 finished with value: 0.8945857011221755 and parameters: {'learning_rate': 0.003914154878049531, 'max_depth': 88, 'num_leaves': 126, 'reg_alpha': 7.060944214852537, 'reg_lambda': 2.7970374277678913, 'colsample_bytree': 0.7563673780575333, 'min_child_samples': 9, 'subsample_freq': 4, 'subsample': 0.5395396760653526, 'min_data_per_group': 128, 'cat_smooth': 83, 'cat_l2': 5}. Best is trial 2 with value: 0.8954543330759283.


Fold 14 | Eval AUC: 0.8950487035787716

------ 15-fold evaluation -----
{'learning_rate': 0.05, 'max_depth': 100, 'num_leaves': 126, 'reg_alpha': 2.5910489528205263, 'reg_lambda': 3.8559682831555833, 'random_state': 2021, 'metric': 'auc', 'n_estimators': 10000, 'n_jobs': -1, 'bagging_seed': 2021, 'feature_fraction_seed': 2021, 'colsample_bytree': 0.7422216044411198, 'min_child_samples': 240, 'subsample_freq': 4, 'subsample': 0.5679619675800118, 'min_data_per_group': 143, 'cat_smooth': 73, 'cat_l2': 19, 'device_type': 'gpu'}

----- FOLD 0 -----
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.894805
[400]	valid_0's auc: 0.895765
Early stopping, best iteration is:
[348]	valid_0's auc: 0.895839
Fold 0 | Eval AUC: 0.8958392469428444

----- FOLD 1 -----
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.897556
[400]	valid_0's auc: 0.89823
Early stopping, best iteration is:
[413]	valid_0's auc: 0.898287
Fold 1 | Eval AUC: 

[I 2021-04-14 02:21:29,404] Trial 4 finished with value: 0.8948644485196914 and parameters: {'learning_rate': 0.005322517892558813, 'max_depth': 100, 'num_leaves': 126, 'reg_alpha': 2.5910489528205263, 'reg_lambda': 3.8559682831555833, 'colsample_bytree': 0.7422216044411198, 'min_child_samples': 240, 'subsample_freq': 4, 'subsample': 0.5679619675800118, 'min_data_per_group': 143, 'cat_smooth': 73, 'cat_l2': 19}. Best is trial 2 with value: 0.8954543330759283.


Fold 14 | Eval AUC: 0.8961509719275591


In [34]:
# Best score
study.best_value

0.8954543330759283

In [35]:
# Historic
plot_optimization_history(study)

In [36]:
#plot_parallel_coordinate: interactively visualizes the hyperparameters and scores
optuna.visualization.plot_parallel_coordinate(study)

In [37]:
plot_param_importances(study)

In [38]:
# plot_slice: shows the evolution of the search. You can see where in the hyperparameter space your search
# went and which parts of the space were explored more.
optuna.visualization.plot_slice(study)

In [39]:
# Best parameters
BEST_PARAMS.update(study.best_params)
BEST_PARAMS

{'n_estimators': 10000,
 'learning_rate': 0.006112383166083743,
 'metric': 'auc',
 'device_type': 'gpu',
 'max_depth': 107,
 'num_leaves': 106,
 'reg_alpha': 8.587061211164151,
 'reg_lambda': 1.6221116007938745,
 'colsample_bytree': 0.3092809118796852,
 'min_child_samples': 53,
 'subsample_freq': 9,
 'subsample': 0.577468721340529,
 'min_data_per_group': 87,
 'cat_smooth': 75,
 'cat_l2': 8}

# 6. Inference

In [40]:
# Update hyperparams for prediction
BEST_PARAMS['learning_rate'] = 0.03238848685934311

In [41]:
# Finally predictions on test set and submission using all the hypertuned parameters and relevant dataframe
test['target'] = kfold_prediction(X, Y, X_TEST, K, BEST_PARAMS, 500)
test['target'].to_csv('submission.csv')


------ 15-fold evaluation -----
{'n_estimators': 10000, 'learning_rate': 0.03238848685934311, 'metric': 'auc', 'device_type': 'gpu', 'max_depth': 107, 'num_leaves': 106, 'reg_alpha': 8.587061211164151, 'reg_lambda': 1.6221116007938745, 'colsample_bytree': 0.3092809118796852, 'min_child_samples': 53, 'subsample_freq': 9, 'subsample': 0.577468721340529, 'min_data_per_group': 87, 'cat_smooth': 75, 'cat_l2': 8}

----- FOLD 0 -----
Training until validation scores don't improve for 500 rounds
[200]	valid_0's auc: 0.89307
[400]	valid_0's auc: 0.895722
[600]	valid_0's auc: 0.896527
[800]	valid_0's auc: 0.896739
[1000]	valid_0's auc: 0.896755
[1200]	valid_0's auc: 0.896676
[1400]	valid_0's auc: 0.8966
Early stopping, best iteration is:
[908]	valid_0's auc: 0.896786

----- FOLD 1 -----
Training until validation scores don't improve for 500 rounds
[200]	valid_0's auc: 0.895705
[400]	valid_0's auc: 0.898731
[600]	valid_0's auc: 0.89969
[800]	valid_0's auc: 0.899931
[1000]	valid_0's auc: 0.900137

In [42]:
end_time = datetime.datetime.now()
print(end_time - start_time)

7:47:37.068190
